## NER Extractor (Synthetic BIO)

Bu notebook 2-ci mərhələni (Extracting) göstərir.

Guardrail (classifier) bir mesajı UNSAFE kimi işarələyirsə, burada işə düşən NER modeli mesajın içindəki PII hissələrini tapır və sonradan həmin hissələri `****` ilə maskalamağa imkan verir.

#### Niyə sintetik data?
Çünki FİN və telefon formatları bizdə spesifikdir və real NER data toplamaq çətin ola bilər. Ona görə əvvəlcə sintetik BIO dataset ilə pipeline-i oturduruq.

In [15]:
from pathlib import Path
import os, sys, subprocess

Notebook-lar `task3/notebooks/` içində olacaq. Repo root-a çıxırıq.

In [2]:
ROOT = Path.cwd()
if ROOT.name.lower() == "notebooks":
    ROOT = ROOT.parent

os.chdir(ROOT)
print("Repo root:", Path.cwd())

Repo root: d:\github_repos\kontakt_home_task\task3


Script və modul run-ları üçün ən stabil variant: PYTHONPATH-a src əlavə etməkdir

In [3]:
PYTHONPATH = str(Path.cwd() / "src")
ENV = os.environ.copy()
ENV["PYTHONPATH"] = PYTHONPATH + (os.pathsep + ENV["PYTHONPATH"] if ENV.get("PYTHONPATH") else "")

In [4]:
def run(cmd):
    """Always run with the same python as this notebook kernel + correct PYTHONPATH."""
    if isinstance(cmd, str):
        cmd = cmd.split()
    print("RUN:", " ".join(cmd))
    return subprocess.run(cmd, env=ENV, check=True)

## 1) Seqeval dependency

### Niyə `seqeval` lazımdır?

NER üçün “accuracy” təkbaşına çox şey demir. Əsas metriklər:
- precision
- recall
- F1 Score-dur

`evaluate.load("seqeval")` da məhz bu hesablamanı edir. `seqeval` qurulmasa, training zamanı metrik hissəsi error verəcək.

In [5]:
run([sys.executable, "-m", "pip", "install", "-q", "seqeval"])

RUN: d:\github_repos\kontakt_home_task\.venv\Scripts\python.exe -m pip install -q seqeval


CompletedProcess(args=['d:\\github_repos\\kontakt_home_task\\.venv\\Scripts\\python.exe', '-m', 'pip', 'install', '-q', 'seqeval'], returncode=0)

## 2) Generate synthetic data (5k)

In [6]:
run([sys.executable, "scripts/synthetic_ner_generator.py", "--n", "5000"])

RUN: d:\github_repos\kontakt_home_task\.venv\Scripts\python.exe scripts/synthetic_ner_generator.py --n 5000


CompletedProcess(args=['d:\\github_repos\\kontakt_home_task\\.venv\\Scripts\\python.exe', 'scripts/synthetic_ner_generator.py', '--n', '5000'], returncode=0)

## 3) Train NER (1 epoch, batch 128)

In [8]:
run([sys.executable, "-m", "pii_guard.training.train_ner", "--epochs", "1", "--batch", "128", "--max_len", "24"])

RUN: d:\github_repos\kontakt_home_task\.venv\Scripts\python.exe -m pii_guard.training.train_ner --epochs 1 --batch 128 --max_len 24


CompletedProcess(args=['d:\\github_repos\\kontakt_home_task\\.venv\\Scripts\\python.exe', '-m', 'pii_guard.training.train_ner', '--epochs', '1', '--batch', '128', '--max_len', '24'], returncode=0)

## Training parametrləri niyə belədir?

Mənim mühit CPU-ludur, yəni GPU yoxdur.
Ona görə “production-grade training” yox, sürətli iterasiya seçmişəm.
Yəni
- `epochs=1` → sadəcə pipeline-i işlətmək üçün
- `batch=128` → step sayı azalsın deyə
- `max_len=24` → ən böyük sürət qazancı buradan gəlir (time complexity `O(L^2)` olduğu üçün)

Qeyd:
* Əgər `train_ner.py`-də `--max_len` arqumenti əlavə etməmisinizsə, bu komanda error verər.
* O halda ya `--max_len`-i silin, ya da `train_ner.py`-də tokenizer çağırışına `max_length=args.max_len` əlavə edin.

## 4) Minimal masking demo (PyTorch)

In [9]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

In [10]:
LABELS = [
    "O",
    "B-FIN", "I-FIN",
    "B-CARD", "I-CARD",
    "B-PHONE", "I-PHONE",
    "B-PERSON", "I-PERSON",
]
ID2LABEL = {i: l for i, l in enumerate(LABELS)}

In [11]:
tok = AutoTokenizer.from_pretrained("models/ner/pytorch", use_fast=True)
model = AutoModelForTokenClassification.from_pretrained("models/ner/pytorch")
model.eval()

The tokenizer you are loading from 'models/ner/pytorch' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
  

## Demo hissəsi nəyi göstərir?

Burada real sistem kimi “maskalama”nı göstəririk:

1) mətn tokenləşir (offset-lər götürülür)
2) model hər token üçün label proqnozlaşdırır
3) BIO label-lardan spanlar yığılır (məs: PHONE, FIN və s.)
4) həmin spanlar `****` ilə əvəzlənir

Bu demo sadəcə “gözlə görülən nəticə” üçündür. Təbii ki production-da əlavə qaydalar (edge-case-lər, overlap, false positive-lər) tələb olunur.

In [12]:
text = "Mənim adım Elvin Əliyevdir, FİN 94FMDDD və telefon +994 50 123 45 67."
enc = tok(text, return_tensors="pt", truncation=True, max_length=96, return_offsets_mapping=True)
offsets = enc.pop("offset_mapping")[0].tolist()

with torch.no_grad():
    logits = model(**enc).logits[0].cpu().numpy()

In [13]:
pred_ids = logits.argmax(axis=-1).tolist()

spans = []
current = None
for pid, (s, e) in zip(pred_ids, offsets):
    if s == 0 and e == 0:
        continue
    lab = ID2LABEL.get(pid, "O")
    if lab == "O":
        if current:
            spans.append(current); current = None
        continue
    if lab.startswith("B-"):
        if current:
            spans.append(current)
        current = (lab[2:], s, e)
    else:  # I-
        ent = lab[2:]
        if current and current[0] == ent:
            current = (current[0], current[1], e)
        else:
            if current:
                spans.append(current)
            current = (ent, s, e)

if current:
    spans.append(current)

In [14]:
spans.sort(key=lambda x: x[1])

masked = []
last = 0
for ent, s, e in spans:
    masked.append(text[last:s])
    masked.append("****")
    last = e
masked.append(text[last:])

print("TEXT  :", text)
print("SPANS :", spans)
print("MASKED:", "".join(masked))

TEXT  : Mənim adım Elvin Əliyevdir, FİN 94FMDDD və telefon +994 50 123 45 67.
SPANS : []
MASKED: Mənim adım Elvin Əliyevdir, FİN 94FMDDD və telefon +994 50 123 45 67.


### Qısa qeydlər (real life olsaydı nələr dəyişə bilərdi?)

- Token-based NER bəzən sərhədləri tam tutmaya bilər (xüsusilə adlar və qarışıq cümlələr)
- Ona görə praktikada çox vaxt **NER + regex** birlikdə işlədilir:
  - regex: FIN / PHONE / CARD kimi “formatı dəqiq” şeylər
  - NER: PERSON kimi daha “yumşaq” entity-lər

Bu taskda da məqsəd cascading arxitekturanın işlədiyini və optimizasiyaya hazır olduğunu göstərməkdir.
